<img src="notes/object1.png" style="height:400px">
<img src="notes/object2.png" style="height:400px">
<img src="notes/object3.png" style="height:400px">
<img src="notes/object4.png" style="height:400px">
<img src="notes/ort1.png" style="height:250px">
<img src="notes/ort2.png" style="height:250px">
<img src="notes/ort3.png" style="height:250px">
<img src="notes/ort4.png" style="height:250px">
<img src="notes/ort5.png" style="height:250px">
<img src="notes/ort6.png" style="height:250px">
<img src="notes/ort7.png" style="height:250px">

In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)

### Note
- While döngüsünün dışında yalnızca bir kere yüz tespiti yapılır. İçinde yapılsaydı bu **nesne takibi** değil, **nesne takibi** olacaktı.
- While döngüsünde artık **nesne takibi** yapılacaktır.

In [3]:
# read one frame only once.
ret, frame = cap.read()

if ret == False:
    print("Warning!")

In [4]:
# face detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")    
face_rectangles = face_cascade.detectMultiScale(frame)

In [5]:
(face_x, face_y, w, h) = tuple(face_rectangles[0]) # there is one rectangle.
track_window = (face_x, face_y, w, h) # meanshift algoritması girdisi

In [6]:
# region of interest (rectangle'ın içi yani ronaldonun yüzü)
roi = frame[face_y:face_y + h, face_x : face_x + w] # roi = face

In [7]:
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

In [8]:
roi_hist = cv2.calcHist([hsv_roi],[0], None, [180], [0,180]) # takip için histogram gerekli
cv2.normalize(roi_hist , roi_hist, 0, 255, cv2.NORM_MINMAX) # normalize historgram in 0-255

# takip icin gerekli durdurma kriterleri
# count = hesaplanacak maksimum oge sayısı
# eps = degisiklik
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1)

In [9]:
while True: # started tracking
    
    ret, frame = cap.read()
    
    if ret:
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # histogramı bir goruntude bulmak için kullnıyoruz (backpropagation)
        # piksel karşılaştırma
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180],1)

        ret, track_window = cv2.meanShift(dst, track_window, term_crit)
        
        x,y,w,h = track_window
        
        img2 = cv2.rectangle(frame, (x,y), (x+w, y+h),(0, 0, 255), 5)
        
        cv2.imshow("Tracking", img2)
        
        if cv2.waitKey(1) == ord("q"): break
            
cap.release()
cv2.destroyAllWindows()